# Running simulations through the cloud 

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
import tidy3d as td
import tidy3d.web as web

# set logging level to ERROR because we'll only running simulations to demonstrate the web API, we dont care about the results.
td.config.logging_level = "ERROR"


## Setup

Let's set up a simple simulation to get started.

In [2]:
# whether to print output in web functions, note: if False (default) they will all run silently
verbose = True

# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1 / fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]), medium=dielectric
)

# create source
source = td.UniformCurrentSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=fwidth),
    polarization="Ex",
)

# create monitor
monitor = td.FieldMonitor(
    fields=["Ex", "Ey", "Ez"],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="field",
)

# Initialize simulation
sim = td.Simulation(
    size=sim_size,
    grid_spec=td.GridSpec.uniform(dl),
    structures=[square],
    sources=[source],
    monitors=[monitor],
    run_time=run_time,
    boundary_spec=td.BoundarySpec.all_sides(boundary=pml),
)


## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
# upload the simulation to our servers
task_id = web.upload(sim, task_name="webAPI", verbose=verbose)

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id, verbose=verbose)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path="data/sim.hdf5", verbose=verbose)


[19:29:04] Created task 'webAPI' with task_id 'fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1'.          ]8;id=386758;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=437185;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=554146;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=693792;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=507359;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=501657;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\taskId]8;;\]8;id=507359;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\=]8;;\]8;id=585186;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\fdve]8;;\]8;id=507359;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\-cfcd2744-05a1-4134-87dd-49987dddd85]8;;\              
           ]8;id=507359;https://tidy3d.simulation.cloud/workbench?taskId=fdve-cfcd2744-05a1-4134-87dd-49987dddd85av1\av1']8;;\.                                                                                                   

Output()

[19:29:06] status = queued                                                                            ]8;id=503535;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=504973;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#321\321]8;;\

Output()

...

[19:29:17] status = postprocess                                                                       ]8;id=165820;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=207653;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#383\383]8;;\

Output()

[19:29:20] status = success                                                                           ]8;id=359326;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=333129;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#390\390]8;;\

Output()

[19:29:21] loading SimulationData from data/sim.hdf5                                                  ]8;id=504759;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=285527;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the [web.run()](../_autosummary/tidy3d.web.run.html) function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to [web.start](../_autosummary/tidy3d.web.start.html) and then exit the session and load the results at a later time using [web.load](../_autosummary/tidy3d.web.load.html).


## Job Container

The [web.Job](../_autosummary/tidy3d.web.Job.html) interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original [Simulation](../_autosummary/tidy3d.Simulation.html).

We can get the cost estimate of running the task before actually running it. This prevents us from accidentally running large jobs that we set up by mistake. The estimated cost is the maximum cost corresponding to running all the time steps.

In [4]:
# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name="job", verbose=verbose)

# estimate the maximum cost
estimated_cost = web.estimate_cost(job.task_id)

print(f'The estimated maximum cost is {estimated_cost:.3f} Flex Credits.')

[19:29:21] Created task 'job' with task_id 'fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1'.             ]8;id=110694;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=465419;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=108317;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=550569;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=729382;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=82085;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\taskId]8;;\]8;id=729382;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\=]8;;\]8;id=974044;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\fdve]8;;\]8;id=729382;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\-b4fbbc51-a302-465e-b1a7-27307447ea5]8;;\              
           ]8;id=729382;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b4fbbc51-a302-465e-b1a7-27307447ea5ev1\ev1']8;;\.                                                                                                   

Output()

The estimated maximum cost is 0.025 Flex Credits.


While [Job](../_autosummary/tidy3d.web.Job.html) has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [5]:
# start job, monitor, and load results
sim_data = job.run(path="data/sim.hdf5")

[19:29:27] status = queued                                                                            ]8;id=572064;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=556214;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#321\321]8;;\

Output()

[19:29:29] status = preprocess                                                                        ]8;id=260539;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=954088;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#315\315]8;;\

[19:29:32] Maximum FlexCredit cost: 0.025. Use 'web.real_cost(task_id)' to get the billed FlexCredit  ]8;id=701155;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=560260;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#338\338]8;;\
           cost after a simulation run.                                                                            

           starting up solver                                                                         ]8;id=736366;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=787494;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#342\342]8;;\

...

[19:29:38] status = postprocess                                                                       ]8;id=920298;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=963201;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#383\383]8;;\

Output()

[19:29:40] status = success                                                                           ]8;id=834022;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=843080;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#390\390]8;;\

Output()

[19:29:42] loading SimulationData from data/sim.hdf5                                                  ]8;id=75426;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=927339;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

Another convenient thing about [Job](../_autosummary/tidy3d.web.Job.html) objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [6]:
# saves the job metadata to a single file
job.to_file("data/job.json")

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file("data/job.json")

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path="data/sim.hdf5")


Output()

[19:29:43] loading SimulationData from data/sim.hdf5                                                  ]8;id=308847;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=852946;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in [Batch](../_autosummary/tidy3d.web.Batch.html) object.
The batch object is like a [Job](../_autosummary/tidy3d.web.Job.html), but stores task metadata for a series of simulations.

In [7]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f"sim_{i}": sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims, verbose=verbose)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir="data")


[19:29:43] Created task 'sim_0' with task_id 'fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1'.           ]8;id=869608;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=61597;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=580605;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=85172;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=992192;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=447511;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\taskId]8;;\]8;id=992192;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\=]8;;\]8;id=153319;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\fdve]8;;\]8;id=992192;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\-9e673420-4d5f-4a3e-a904-656d635f516]8;;\              
           ]8;id=992192;https://tidy3d.simulation.cloud/workbench?taskId=fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1\7v1']8;;\.                                                                                                   

Output()

[19:29:44] Created task 'sim_1' with task_id 'fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1'.           ]8;id=648007;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=891905;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=548236;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=27139;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=816219;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=664414;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\taskId]8;;\]8;id=816219;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\=]8;;\]8;id=586674;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\fdve]8;;\]8;id=816219;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\-8b06cd30-03a0-47e8-a5f8-d1ceaac37d1]8;;\              
           ]8;id=816219;https://tidy3d.simulation.cloud/workbench?taskId=fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1\5v1']8;;\.                                                                                                   

...

           View task using web UI at                                                                  ]8;id=234165;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=702486;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=781229;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=304978;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\taskId]8;;\]8;id=781229;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\=]8;;\]8;id=259062;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\fdve]8;;\]8;id=781229;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\-2fc1be20-b997-416e-97fa-61092721c5a]8;;\              
           ]8;id=781229;https://tidy3d.simulation.cloud/workbench?taskId=fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1\0v1']8;;\.                                                                                                   

Output()

[19:29:47] Started working on Batch.                                                               ]8;id=817154;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py\container.py]8;;\:]8;id=462859;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py#457\457]8;;\

Output()

[19:30:08] Batch complete.                                                                         ]8;id=488928;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py\container.py]8;;\:]8;id=222288;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py#497\497]8;;\

When the batch is completed, the output is not a [SimulationData](../_autosummary/tidy3d.SimulationData.html) but rather a [BatchData](../_autosummary/tidy3d.web.BatchData.html).  The data within this [BatchData](../_autosummary/tidy3d.web.BatchData.html) object can either be indexed directly `batch_results[task_name]` or can be looped through `batch_results.items()` to get the [SimulationData](../_autosummary/tidy3d.SimulationData.html) for each task.

This was chosen to reduce the memory strain from loading all [SimulationData](../_autosummary/tidy3d.SimulationData.html) objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [8]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[19:30:10] loading SimulationData from data\fdve-9e673420-4d5f-4a3e-a904-656d635f5167v1.hdf5          ]8;id=889503;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=760994;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

Output()

[19:30:11] loading SimulationData from data\fdve-8b06cd30-03a0-47e8-a5f8-d1ceaac37d15v1.hdf5          ]8;id=180757;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=916590;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

Output()

[19:30:13] loading SimulationData from data\fdve-2fc1be20-b997-416e-97fa-61092721c5a0v1.hdf5          ]8;id=672407;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=310110;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

{'sim_0': 13540217.220344715, 'sim_1': 13540217.220344715, 'sim_2': 13540217.220344715}


## Asynchronous Batching

Finally, one can make use of the [asyncio package](https://realpython.com/async-io-python/) to perform asynchronous processing of several simulations.

For this purpose, a [web.run_async](../_autosummary/tidy3d.web.run_async.html) function is provided, which works like the regular `web.run` but accepts a dictionary of simulations. 

Here is the previous example repeated using this feature.

In [9]:
batch_results = web.run_async(simulations=sims, verbose=verbose)


[19:30:13] Created task 'sim_0' with task_id 'fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1'.           ]8;id=487643;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=450105;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=365999;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=379828;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=843831;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=674969;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\taskId]8;;\]8;id=843831;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\=]8;;\]8;id=941731;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\fdve]8;;\]8;id=843831;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\-e8e5173c-32f0-481f-a2a5-e7b30269b49]8;;\              
           ]8;id=843831;https://tidy3d.simulation.cloud/workbench?taskId=fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1\7v1']8;;\.                                                                                                   

Output()

[19:30:14] Created task 'sim_1' with task_id 'fdve-b1eb61e8-9429-484c-b791-86c62b839037v1'.           ]8;id=207952;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=146171;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#186\186]8;;\

           View task using web UI at                                                                  ]8;id=905410;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=519731;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=144207;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=925841;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\taskId]8;;\]8;id=144207;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\=]8;;\]8;id=242658;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\fdve]8;;\]8;id=144207;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\-b1eb61e8-9429-484c-b791-86c62b83903]8;;\              
           ]8;id=144207;https://tidy3d.simulation.cloud/workbench?taskId=fdve-b1eb61e8-9429-484c-b791-86c62b839037v1\7v1']8;;\.                                                                                                   

...

           View task using web UI at                                                                  ]8;id=290329;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=500599;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#188\188]8;;\
           ]8;id=732446;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=617932;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\taskId]8;;\]8;id=732446;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\=]8;;\]8;id=767493;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\fdve]8;;\]8;id=732446;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\-717727ca-9a7c-4243-9c13-5523a3fa402]8;;\              
           ]8;id=732446;https://tidy3d.simulation.cloud/workbench?taskId=fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1\bv1']8;;\.                                                                                                   

Output()

[19:30:20] Started working on Batch.                                                               ]8;id=200149;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py\container.py]8;;\:]8;id=112227;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py#457\457]8;;\

Output()

[19:30:40] Batch complete.                                                                         ]8;id=825027;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py\container.py]8;;\:]8;id=635194;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\container.py#497\497]8;;\

In [10]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch_results.items():
    intensity = sim_data.get_intensity("field").sel(f=freq0)
    sum_intensity = float(intensity.sum(("x", "y")).values)
    intensities[task_name] = sum_intensity

print(intensities)


Output()

[19:30:42] loading SimulationData from .\fdve-e8e5173c-32f0-481f-a2a5-e7b30269b497v1.hdf5             ]8;id=40583;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=130183;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

Output()

[19:30:43] loading SimulationData from .\fdve-b1eb61e8-9429-484c-b791-86c62b839037v1.hdf5             ]8;id=521403;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=1658;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

Output()

[19:30:45] loading SimulationData from .\fdve-717727ca-9a7c-4243-9c13-5523a3fa402bv1.hdf5             ]8;id=611042;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py\webapi.py]8;;\:]8;id=224027;file://C:\Users\xinzhong\anaconda3\envs\tidy3d_env\lib\site-packages\tidy3d\web\webapi.py#568\568]8;;\

{'sim_0': 13540217.220344715, 'sim_1': 13540217.220344715, 'sim_2': 13540217.220344715}
